# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 5/24/2022 6:46:09 AM


In [2]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
using BoSSS.Application.SipPoisson;

In [4]:
wmg.Init("WIP-k-LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'WIP-k-LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer'.


In [5]:
ExecutionQueues

index,RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
0,win\amd64,C:\BoSSStests,False,MSHPC-Gitrunner-HighPrio,dotnet,FDY\jenkinsci,DC2,[ fdygitrunner ],Highest,True,"[ C:\BoSSStests == , C:\Users\jenkinsci\ == ]"
1,win\amd64,\\fdygitrunner\BoSSStests,False,MSHPC-Gitrunner-DefaultTest,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, fdygitrunner, hpcluster3, hpccluster2 ]",Normal,True,[ \\fdygitrunner\BoSSStests == ]
2,win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2, hpcluster3, fdygitrunner ]",Normal,True,[ \\fdygitrunner\ValidationTests == ]
3,win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-FastNodes,dotnet,FDY\jenkinsci,DC2,[ hpcluster3 ],Normal,True,[ \\fdygitrunner\ValidationTests == ]


Queue which will be used:

In [6]:
GetDefaultQueue()

RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2, hpcluster3, fdygitrunner ]",Normal,True,[ \\fdygitrunner\ValidationTests == ]


## Utility definitions

In [7]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [8]:
wmg.Grids

Opening existing database 'C:\BoSSStests\WIP-k-LinslvPerfSer'.


#0: { Guid = 594ff2ad-dc94-481e-807b-31f46e1519cd; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J1310720; Cell Count = 1310720; Dim = 3 }
#1: { Guid = 735a9e2d-67ce-41b8-aa97-d38ce827238c; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J552960; Cell Count = 552960; Dim = 3 }
#2: { Guid = 5f9955a3-954e-44e5-ae82-5f7e451b5c10; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J163840; Cell Count = 163840; Dim = 3 }
#3: { Guid = ee953445-5c67-44c5-8ec9-8a4a70f92431; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = 6162fb52-04e2-4fa2-a719-cd2d4d4d8ee7; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J20480; Cell Count = 20480; Dim = 3 }
#5: { Guid = 2cb2c452-9e53-4aee-863f-e4ffcf87da43; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J2560; Cell Count = 2560; Dim = 3 }
#6: { Guid = 43a10f77-866f-4a1d-a06e-99ee6bcc0617; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J320; Cell Count = 320; Dim = 3 }


In [9]:
//wmg.Grids.Skip(17).ForEach(g => g.Delete(true));

Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [10]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = wmg.CurrentProject + "-SIP_Poisson_J" + J;
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Found Grid: { Guid = 43a10f77-866f-4a1d-a06e-99ee6bcc0617; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J320; Cell Count = 320; Dim = 3 }
Found Grid: { Guid = 2cb2c452-9e53-4aee-863f-e4ffcf87da43; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J2560; Cell Count = 2560; Dim = 3 }
Found Grid: { Guid = 6162fb52-04e2-4fa2-a719-cd2d4d4d8ee7; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J20480; Cell Count = 20480; Dim = 3 }
Found Grid: { Guid = ee953445-5c67-44c5-8ec9-8a4a70f92431; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J69120; Cell Count = 69120; Dim = 3 }
Found Grid: { Guid = 5f9955a3-954e-44e5-ae82-5f7e451b5c10; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J163840; Cell Count = 163840; Dim = 3 }
Found Grid: { Guid = 735a9e2d-67ce-41b8-aa97-d38ce827238c; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J552960; Cell Count = 552960; Dim = 3 }
Found Grid: { Guid = 594ff2ad-dc94-481e-807b-31f46e1519cd; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J1310720; Cell Count = 1310720; Dim = 3 }


In [11]:
grids

#0: { Guid = 43a10f77-866f-4a1d-a06e-99ee6bcc0617; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J320; Cell Count = 320; Dim = 3 }
#1: { Guid = 2cb2c452-9e53-4aee-863f-e4ffcf87da43; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J2560; Cell Count = 2560; Dim = 3 }
#2: { Guid = 6162fb52-04e2-4fa2-a719-cd2d4d4d8ee7; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J20480; Cell Count = 20480; Dim = 3 }
#3: { Guid = ee953445-5c67-44c5-8ec9-8a4a70f92431; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = 5f9955a3-954e-44e5-ae82-5f7e451b5c10; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J163840; Cell Count = 163840; Dim = 3 }
#5: { Guid = 735a9e2d-67ce-41b8-aa97-d38ce827238c; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J552960; Cell Count = 552960; Dim = 3 }
#6: { Guid = 594ff2ad-dc94-481e-807b-31f46e1519cd; Name = WIP-k-LinslvPerfSer-SIP_Poisson_J1310720; Cell Count = 1310720; Dim = 3 }


## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [12]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [13]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [14]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.19866933079506122

In [15]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.8390715290764524

### Setup of Parameter Study

Polynomial degrees to test:

In [16]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5}; // reduced options for dev.

Solvers which we want to instrument:

In [17]:
using BoSSS.Solution.AdvancedSolvers;

In [18]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso, 
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.pMultigrid
    //LinearSolverCode.exp_Kcycle_ILU
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [19]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 3000000; // Up to 3 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [20]:
List<SipControl> controls = new List<SipControl>();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > GetMaxAllowedDOF(solver_name)) {
        continue;
    } 
    var ctrl = new SipControl();
    controls.Add(ctrl);
       
    
    string caseName = string.Format("Poisson_J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = "SIP_" + caseName;

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver           = solver_name.GetConfig();
    var isc = ctrl.LinearSolver as IterativeSolverConfig;
    if(isc != null) {
        //Console.WriteLine(isc.ConvergenceCriterion);
        //ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
        isc.ConvergenceCriterion = 1e-8;
    }
        
        
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.NoOfMultigridLevels = 100; // maximum number of multigrid levels to use; actual number will be far lower.
    
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.METIS;
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.directHilbert;
}
}
}

setting up: Poisson_J320_k2_direct_pardiso
setting up: Poisson_J2560_k2_direct_pardiso
setting up: Poisson_J20480_k2_direct_pardiso
setting up: Poisson_J69120_k2_direct_pardiso
setting up: Poisson_J320_k3_direct_pardiso
setting up: Poisson_J2560_k3_direct_pardiso
setting up: Poisson_J20480_k3_direct_pardiso
setting up: Poisson_J320_k5_direct_pardiso
setting up: Poisson_J2560_k5_direct_pardiso
setting up: Poisson_J320_k2_exp_gmres_levelpmg
setting up: Poisson_J2560_k2_exp_gmres_levelpmg
setting up: Poisson_J20480_k2_exp_gmres_levelpmg
setting up: Poisson_J69120_k2_exp_gmres_levelpmg
setting up: Poisson_J163840_k2_exp_gmres_levelpmg
setting up: Poisson_J320_k3_exp_gmres_levelpmg
setting up: Poisson_J2560_k3_exp_gmres_levelpmg
setting up: Poisson_J20480_k3_exp_gmres_levelpmg
setting up: Poisson_J69120_k3_exp_gmres_levelpmg
setting up: Poisson_J320_k5_exp_gmres_levelpmg
setting up: Poisson_J2560_k5_exp_gmres_levelpmg
setting up: Poisson_J20480_k5_exp_gmres_levelpmg
setting up: Poisson_J320

In [21]:
/*
// Save results on local workstation to reporduce particular cases:
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\ipPoisson\bin\Release\net5.0\PoissonBench";
foreach(var c in controls) {
    string name = c.SessionName;
    Console.Write(name + ": ");
    try {
        
        var txt = c.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

In [22]:
controls.Count()

45

## Launch Jobs

In [23]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    ctrl.RunBatch(); // run in the default queue for this project
}

 Submitting: SIP_Poisson_J320_k2_direct_pardiso
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J320_k2_direct_pardiso	05/23/2022 17:54:22	803a2124..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_Poisson_J2560_k2_direct_pardiso
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k2_direct_pardiso	05/23/2022 17:54:32	34b95dee..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_Poisson_J20480_k2_direct_pardiso
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J20480_k2_direct_pardiso	05/23/2022 17:54:43	acd0cb48..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_Poisson_J69120_k2_direct_pardiso
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k2_direct_pardiso	05/23/2022 17:54:54	3afd81a

 Submitting: SIP_Poisson_J69120_k3_exp_Kcycle_schwarz
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k3_exp_Kcycle_schwarz	05/24/2022 06:04:22	79f7c40b..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_Poisson_J320_k5_exp_Kcycle_schwarz
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J320_k5_exp_Kcycle_schwarz	05/23/2022 18:00:16	7bfe7fcf..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_Poisson_J2560_k5_exp_Kcycle_schwarz
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k5_exp_Kcycle_schwarz	05/24/2022 06:04:36	89391ec8..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_Poisson_J20480_k5_exp_Kcycle_schwarz
Info: Found successful session "WIP-k-LinslvPerfSer	SIP_Poisson_J20480_k5_exp_Kcycle_sch

In [24]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [25]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

All jobs finished.


In [26]:
wmg.AllJobs

#0: SIP_Poisson_J320_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_Poisson_J320_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: SIP_Poisson_J2560_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_Poisson_J2560_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: SIP_Poisson_J20480_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_Poisson_J20480_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: SIP_Poisson_J69120_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_Poisson_J69120_k2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdyg

In [27]:
wmg.Sessions

#0: WIP-k-LinslvPerfSer	SIP_Poisson_J20480_k5_pMultigrid	05/24/2022 06:05:37	bebf2868...
#1: WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k3_exp_Kcycle_schwarz	05/24/2022 06:04:22	79f7c40b...
#2: WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k3_exp_gmres_levelpmg	05/24/2022 06:03:25	7d30251d...
#3: WIP-k-LinslvPerfSer	SIP_Poisson_J163840_k2_exp_Kcycle_schwarz	05/24/2022 06:03:54	171f6bc0...
#4: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k5_pMultigrid	05/24/2022 06:05:24	c95ff43e...
#5: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k5_exp_Kcycle_schwarz	05/24/2022 06:04:36	89391ec8...
#6: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k3_pMultigrid	05/24/2022 06:04:56	ddc16fa7...
#7: WIP-k-LinslvPerfSer	SIP_Poisson_J320_k5_pMultigrid	05/24/2022 06:05:11	d68fd4b7...
#8: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k3_exp_Kcycle_schwarz	05/24/2022 06:04:08	0b6ec0f3...
#9: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k2_exp_Kcycle_schwarz	05/24/2022 06:03:40	3b511c77...
#10: WIP-k-LinslvPerfSer	SIP_Poisson_J20480_k5_exp_Kcycle_schwarz	05

In [28]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

In [29]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + fail.LatestDeployment.DeploymentDirectory.FullName);

In [30]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [31]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

#0: WIP-k-LinslvPerfSer	SIP_Poisson_J163840_k2_exp_Kcycle_schwarz*	05/23/2022 17:59:13	a66ecfdf...
#1: WIP-k-LinslvPerfSer	SIP_Poisson_J20480_k5_pMultigrid*	05/23/2022 18:03:24	24c953d9...
#2: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k5_pMultigrid*	05/23/2022 18:03:11	d038ae4e...
#3: WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k3_exp_Kcycle_schwarz*	05/23/2022 18:00:04	5918c7ee...
#4: WIP-k-LinslvPerfSer	SIP_Poisson_J320_k5_pMultigrid*	05/23/2022 18:02:58	77ab5e43...
#5: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k3_pMultigrid*	05/23/2022 18:02:18	9d5891d6...
#6: WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k3_exp_gmres_levelpmg*	05/23/2022 17:57:33	86674daf...
#7: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k5_exp_Kcycle_schwarz*	05/23/2022 18:00:29	22980daf...
#8: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k3_exp_Kcycle_schwarz*	05/23/2022 17:59:39	e35953a8...
#9: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k2_exp_Kcycle_schwarz*	05/23/2022 17:58:36	8c191bd0...


In [36]:
wmg.AllJobs["SIP_Poisson_J2560_k5_pMultigrid"].AllDeployments

#0: Job 449852, FailedOrCanceled 'WIP-k-LinslvPerfSer-ipPoisson2022May23_180255' @ MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests
#1: Job 449862, FinishedSuccessful 'WIP-k-LinslvPerfSer-ipPoisson2022May24_060509' @ MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests


In [32]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
//foreach(Job jF in NoSuccess)
//   jF.DeleteOldDeploymentsAndSessions();

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout

## Create Table for Post-Processing

(moved to post-processing worksheet)

In [ ]:
/*
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}
*/

In [ ]:
/*
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
*/

In [ ]:
/*
var FullSessTab = wmg.SessionTable;

// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = FullSessTab.GetColumnNames().ToList();
AllCols.Remove("Session");
FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());
*/

In [ ]:
//foreach(var s in AllCols) Console.WriteLine(s);

In [ ]:
//foreach(var s in wmg.Sessions[0].KeysAndQueries.Keys) Console.WriteLine(s + " \t = " + wmg.Sessions[0].KeysAndQueries[s]);

In [ ]:
/*var SubTab = FullSessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");
*/

In [ ]:
//SubTab.Print();

In [ ]:
/*
// Filename
var now               = DateTime.Now;
FullSessTab.TableName = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath        = FullSessTab.TableName + ".json";
SubTab.SaveToFile(wmg.CurrentProject + ".json");
SubTab.ToCSVFile(wmg.CurrentProject + ".csv");
*/